In [1]:
import sys
sys.path.append('/global/homes/s/sleak/.local/cori/3.6-anaconda-4.4/lib/python3.6/site-packages')
import rdflib

#sys.path.append('/global/homes/s/sleak/Monitoring/Resilience/LogSet/prototypes')
#import Graph    
#
## force re-read of Graph.py in case I've edited it:
#import importlib
#importlib.reload(Graph)
#
#url = 'nersc-catalog.ttl'
#vocab = '../etc/logset.ttl'
##Graph.read(url, vocab=vocab)
#Graph.construct(url, local_only=False)
#
#Graph.dump()
def dump(graph):
    print(graph.serialize(format='n3').decode('ascii'))

import urllib.request
def parse_line_by_line(url, fmt):
    """ rdflib is really uninformative about the location of the error
        when it can't parse a file, so use this to parse line-by-line 
        and look for errors
        (hmm, does not work, fails on valid files)
    """
    f = urllib.request.urlopen(url)
    g = rdflib.ConjunctiveGraph()
    linecount=0
    for line in f.readlines():
        linecount += 1
        print("parsing line {0:d}: {1}".format(linecount,line))
        rdftext = line.strip()
        if not rdftext: continue
        try:
            g.parse(data=rdftext, format=fmt)
        except:
            print("failed to parse line {0:d} of {1}".format(linecount,url))
            print(line)
            print(g.serialize())
            raise
    print("finished parsing {0:s} successfully!".format(url))
    
import os
def construct(*catalog_urls, spider=False):
    graph = rdflib.ConjunctiveGraph()
    unparsed = set(catalog_urls)
    print(unparsed)
    parsed = set()
    q_peers = 'SELECT ?uri WHERE { ?cat logset:peers ?uri . }'
    # get the catalogs:
    while len(unparsed) > 0:
        for url in unparsed:
            fmt = rdflib.util.guess_format(url) 
            print("parsing {0}".format(url))
            graph.parse(url, format=fmt)
            print ("graph now has {0} statements".format(len(graph)))
            parsed.add(url)
        unparsed = set()   
        if spider:
            for peer in graph.query(q_peers):
                url = peer['uri']
                if not url in parsed:
                    unparsed.add(url)
                    
            #graph.parse(peer['uri'], format=rdflib.util.guess_format(peer['uri']))
                    
    # get the LogSets:
    q_logsets = ''' SELECT ?uri 
                    WHERE {
                        ?cat dcat:dataset ?uri . 
                    }'''
    for logset in graph.query(q_logsets):
        print("parsing found logset url: {0}".format(str(logset)))
        url = logset['uri']
        fmt = rdflib.util.guess_format(url)
        graph.parse(url, format=fmt)
        print ("graph now has {0} statements".format(len(graph)))
    return graph

# quick sanity check:
#parse_line_by_line('file:/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/nersc-entities.ttl', 'turtle')
#g1 = rdflib.ConjunctiveGraph().parse("file:/global/u1/s/sleak/Monitoring/Resilience/LogSet/examples/index.ttl", format='turtle')
#print("got this far...")
#print(g1.serialize(format='n3').decode('ascii'))

graph = construct('nersc-catalog.ttl', spider=True)

query = '''SELECT ?id ?what ?path
WHERE {
?id a logset:ConcreteLog . 
?id dcat:downloadURL ?path .
?id logset:isInstanceOf ?what .
} '''

print("concretelogs:")
for row in graph.query(query):
    print(row)

query = '''SELECT ?id ?path
WHERE {
?id a logset:ConcreteLog .
?id logset:isInstanceOf dict:console_logfile . 
?id dcat:downloadURL ?path .
} '''
print("console logfiles:")
for row in graph.query(query):
    print(row)
#dump(graph)
# why am I not finding the console-log that is in the snl index?
# ah, typo in index.ttl, point dict prefix at vocab instead .. fixed now
# first incarnation will prob be rather fragile...
                
    

#[ns for ns in graph.namespaces()]

{'nersc-catalog.ttl'}
parsing nersc-catalog.ttl
graph now has 5 statements
parsing https://raw.githubusercontent.com/NERSC/LogSet/master/examples/snl-catalog.ttl
graph now has 10 statements
parsing https://raw.githubusercontent.com/NERSC/LogSet/master/examples/nersc-catalog.ttl
graph now has 10 statements
parsing found logset url: (rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/examples/index.ttl'),)
graph now has 70 statements
parsing found logset url: (rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/examples/snl-index.ttl'),)
graph now has 95 statements
concretelogs:
(rdflib.term.BNode('f7ca1bfdf005c480099b070c6576fcc5db1'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#console_logfile'), rdflib.term.Literal('snl:p0-20170906t151820/console-20170907'))
(rdflib.term.BNode('fe39eadb7b94f4beeba4c00cf9fe8b756b4'), rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/dict#cons

In [2]:
[ns for ns in graph.namespaces()]

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('dct', rdflib.term.URIRef('http://purl.org/dc/terms/')),
 ('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#')),
 ('entities',
  rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/examples/nersc-entities#')),
 ('logset',
  rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/etc/vocab#')),
 ('',
  rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/examples/nersc-catalog#')),
 ('entities1',
  rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/examples/snl-entities#')),
 ('default1',
  rdflib.term.URIRef('https://raw.githubusercontent.com/NERSC/LogSet/master/examples/snl-catalog#')),
 ('dict',
  rdflib.term.URIR

In [ ]:
# find datasets, read them, and if they are LogSets, add them to the graph: